In [1]:
# Use this initial code to work in the notebook as if it were a module, that 
# is, to be able to export classes and functions from other subpackages.

import os
import sys

package_path = os.path.abspath('.').split(os.sep + 'notebooks')[0]
if package_path not in sys.path:
    sys.path.append(package_path)

%load_ext autoreload
%autoreload 2

In [2]:
from tuyabot_llm import AbsolutePaths
from langchain_community.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

# Load docs

In [3]:
current_dir = AbsolutePaths().get_abs_path_folder('raw')
db_dir = os.path.join(current_dir, "tuya_collection")
persistent_directory_clean = os.path.join(db_dir, "chroma_tuya_collection_cleaned")

In [4]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\mario\AppData\Local\Temp\ipykernel_19204\3367226509.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
db = Chroma(persist_directory=persistent_directory_clean, embedding_function=embeddings)

C:\Users\mario\AppData\Local\Temp\ipykernel_19204\2605525261.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=persistent_directory_clean, embedding_function=embeddings)


# Retrieve docs based in query

In [6]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5},
)

query = "¿Cuáles son los valores la tasa de interés y póliza del producto credicompras?"

relevant_docs = retriever.invoke(query)

print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")


--- Relevant Documents ---
Document 1:
con garantía fondo credicompras dirigido con este crédito puedes comprar productos en los almacenes éxito en las categorías de: electrodomésticos, tecnología, colchones, llantas, muebles, máquinas de ejercicio, bicicletas, motos y movilidad eléctrica. características te ofrecemos montos desde $1.000.000 hasta $15’000,000 y plazos entre 12 y 36 meses. se te hará entrega de una tarjeta de único uso. con ella pagarás tu compra y tendrás 8 días para utilizarla. condiciones del producto. debes usar el monto aprobado en una única transacción de compra. en ella puedes llevar diferentes productos de

Source: https://www.tuya.com.co/credicompras

Document 2:
seguro deudores** $3.143 credicompras libre destino tasa efectiva anual 24,89805% póliza colectiva seguro deudores** $3.143 credicompras bajo monto tasa efectiva anual 56,08602% póliza colectiva seguro deudores $3.143 ** para créditos desembolsados a partir del 1 de abril de 2019. póliza colectiva seg

In [7]:
[i.page_content for i in relevant_docs]

['con garantía fondo credicompras dirigido con este crédito puedes comprar productos en los almacenes éxito en las categorías de: electrodomésticos, tecnología, colchones, llantas, muebles, máquinas de ejercicio, bicicletas, motos y movilidad eléctrica. características te ofrecemos montos desde $1.000.000 hasta $15’000,000 y plazos entre 12 y 36 meses. se te hará entrega de una tarjeta de único uso. con ella pagarás tu compra y tendrás 8 días para utilizarla. condiciones del producto. debes usar el monto aprobado en una única transacción de compra. en ella puedes llevar diferentes productos de',
 'seguro deudores** $3.143 credicompras libre destino tasa efectiva anual 24,89805% póliza colectiva seguro deudores** $3.143 credicompras bajo monto tasa efectiva anual 56,08602% póliza colectiva seguro deudores $3.143 ** para créditos desembolsados a partir del 1 de abril de 2019. póliza colectiva seguro deudores para créditos desembolsados hasta 31 de marzo de 2019: $3.159 por millón desembo

In [8]:
def format_docs(docs):
    formatted_docs = "\n".join(doc.page_content for doc in docs)
    # print(f"**Formatted Docs**: {formatted_docs}\n*******************************")  # Inspeccionar la salida de format_docs
    return formatted_docs

context = format_docs(relevant_docs)
print("Context for LLM:\n\n", context)

Context for LLM:

 con garantía fondo credicompras dirigido con este crédito puedes comprar productos en los almacenes éxito en las categorías de: electrodomésticos, tecnología, colchones, llantas, muebles, máquinas de ejercicio, bicicletas, motos y movilidad eléctrica. características te ofrecemos montos desde $1.000.000 hasta $15’000,000 y plazos entre 12 y 36 meses. se te hará entrega de una tarjeta de único uso. con ella pagarás tu compra y tendrás 8 días para utilizarla. condiciones del producto. debes usar el monto aprobado en una única transacción de compra. en ella puedes llevar diferentes productos de
seguro deudores** $3.143 credicompras libre destino tasa efectiva anual 24,89805% póliza colectiva seguro deudores** $3.143 credicompras bajo monto tasa efectiva anual 56,08602% póliza colectiva seguro deudores $3.143 ** para créditos desembolsados a partir del 1 de abril de 2019. póliza colectiva seguro deudores para créditos desembolsados hasta 31 de marzo de 2019: $3.159 por m

In [9]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5},
)

query = "¿Qué tipos de tarjeta de crédito ofrece TUYA?"

relevant_docs = retriever.invoke(query)

print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")


--- Relevant Documents ---
Document 1:
y te das esos gusticos que siempre has querido. *aplican términos y condiciones de cada tarjeta disponible. adquiere historial crediticio con nuestras tarjetas de crédito tuya para aquellas personas jóvenes o que aún no tienen historial crediticio, solicitar una de nuestras tarjetas de crédito es una excelente oportunidad para ir construyendo su historial de compras a crédito y así acceder más adelante a otros productos crediticios con mayor facilidad. puedes empezar con alguna de nuestras tarjetas de crédito básicas como la tarjeta éxito y realizar compras a una sola cuota para no afectar

Source: https://www.tuya.com.co/tarjetas-de-credito

Document 2:
ti: inicia tu historial crediticio con nuestras tarjetas de crédito. sólo pagas la cuota de manejo cuando usas las tarjetas*. recibes descuentos exclusivos en almacenes éxito y carulla. acumulas puntos colombia para redimir en comercios aliados. descuentos con tarjetas de crédito tuya con nuestra

In [10]:
context = format_docs(relevant_docs)
print("Context for LLM:\n\n", context)

Context for LLM:

 y te das esos gusticos que siempre has querido. *aplican términos y condiciones de cada tarjeta disponible. adquiere historial crediticio con nuestras tarjetas de crédito tuya para aquellas personas jóvenes o que aún no tienen historial crediticio, solicitar una de nuestras tarjetas de crédito es una excelente oportunidad para ir construyendo su historial de compras a crédito y así acceder más adelante a otros productos crediticios con mayor facilidad. puedes empezar con alguna de nuestras tarjetas de crédito básicas como la tarjeta éxito y realizar compras a una sola cuota para no afectar
ti: inicia tu historial crediticio con nuestras tarjetas de crédito. sólo pagas la cuota de manejo cuando usas las tarjetas*. recibes descuentos exclusivos en almacenes éxito y carulla. acumulas puntos colombia para redimir en comercios aliados. descuentos con tarjetas de crédito tuya con nuestras tarjetas de crédito obtienes descuentos exclusivos en los almacenes éxito y carulla. 

# Load model

In [11]:
import torch

if torch.cuda.is_available():
    print("CUDA está disponible. El modelo puede ejecutarse en GPU.")
else:
    print("CUDA no está disponible. El modelo se ejecutará en CPU.")

CUDA está disponible. El modelo puede ejecutarse en GPU.


In [12]:
torch.set_default_tensor_type("torch.cuda.FloatTensor")

c:\Users\mario\Documents\job_interviews\2025_tuya\tuyabot_llm\.venv2\lib\site-packages\torch\__init__.py:1236: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\tensor\python_tensor.cpp:436.)
  _C._set_default_tensor_type(t)


In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline 
from langchain_huggingface import HuggingFacePipeline

model_id = 'unsloth/Llama-3.2-1B-Instruct' 
tokenizer = AutoTokenizer.from_pretrained(model_id, device="cuda:0", truncation=True)
model = AutoModelForCausalLM.from_pretrained(model_id) 


pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2000,
    # truncate=True,
    temperature=0.2,
    top_k=10,
    # repetition_penalty=1.5,
    # no_repeat_ngram_size=4,  # Ajusta el tamaño de los n-gramas que no se pueden repetir
)

#conversión a uso api tipo langchain 
local_llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


# Use RAG

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """
Eres un agente de servicio al cliente que trabaja para TUYA SA, una empresa que se
dedica a ser la solución financiera del retail y que busca apoyar a los sectores
vulnerables de la sociedad. 

Como agente de servicio al cliente, debes suministrar respuestas amigables y
claras a los clientes.

Emplea el contexto que te ofrece la empresa TUYA delimitado por triple comillas invertidas, para responder
la pregunta que se encuentra al final delimitada por comillas simples.

Siempre que puedas responder con una serie de items hazlo, tu respuesta es máximo de 15 palabras y debe ser completamente 
en español.

Contexto: ```{context}```

Pregunta: '{question}'
"""

prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | local_llm
    | StrOutputParser()
)

In [20]:
q = "¿Cuáles son los valores de la tasa de interés y póliza del producto credicompras?"

In [21]:
# rag_chain.invoke(q)

In [24]:
for chunk in rag_chain.stream(q):
    print(chunk, end="", flush=True)

Respuesta: 
- El valor de la tasa de interés es el 24,89805% anual.
- El valor de la póliza de seguro deudores es el 56,08602% anual.

In [33]:
q = """
¿Qué tipos de tarjeta de crédito ofrece TUYA?, enlístalas
"""

for chunk in rag_chain.stream(q):
    print(chunk, end="", flush=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Respuesta: 
Tuya ofrece varias opciones de tarjetas de crédito, incluyendo la tarjeta Success y la Success Gold Mastercard, así como la Success Pro Mastercard. Estas tarjetas ofrecen diferentes beneficios y características, como descuentos exclusivos, acumulación de puntos, y acceso a servicios de pago en línea. Es importante tener en cuenta que cada tarjeta tiene sus propias condiciones y términos, por lo que es recomendable consultar con un representante de Tuya para obtener más información.

# Gradio DEMO

## First demo

In [3]:
# Definir la función para generar la respuesta
def generate_response(question):

    template = """
        Eres un agente de servicio al cliente que trabaja para TUYA SA, una empresa que se
        dedica a ser la solución financiera del retail y que busca apoyar a los sectores
        vulnerables de la sociedad. 

        Como agente de servicio al cliente, debes suministrar respuestas amigables y
        claras a los clientes.

        Emplea el contexto que te ofrece la empresa TUYA delimitado por triple comillas invertidas, para responder
        la pregunta que se encuentra al final delimitada por comillas simples.

        Siempre que puedas responder con una serie de items hazlo, tu respuesta es máximo de 15 palabras y debe ser completamente 
        en español.

        Contexto: ```{context}```

        Pregunta: '{question}'
        """
    
    prompt = PromptTemplate.from_template(template)

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | local_llm
        | StrOutputParser()
    )

    output = rag_chain.invoke(question)

    
    return  output



In [16]:
import gradio as gr

demo = gr.Interface(
    fn=generate_response,
    inputs=[
        gr.Textbox(label="question"),
    ],
    outputs=gr.Textbox(label="answer"),
    title="Agente de Servicio al Cliente TUYABOT",
    description="Pregúntame algo sobre TUYA SA",
)

In [17]:
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://2ec79d8407ccb9ee48.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [37]:
demo.close()

Closing server running on port: 7860
